# Atmospheric Rivers: explore spatiotemporal relationship with avalanches 

This notebook will import the revised avalanche database table, a reference point layer, and the AR events shapefile. A spatiotemporal relationship is explored.
  
  

_Note: the avalanche database table was revised to remove duplicate events, and to choose the nearest populated place to represent the avalanche location. This allows simple linking to a reference point layer, avoiding manual population of coordinates for avalanche events. This method should not produce appreciable errors when considering the scale of this analysis. If the results table will be further filtered by proximity, consider using a value that respects the spatial ambiguity of both the modeled AR boundaries and the generalized avalanche locations._
  
  

For each avalanche event, AR landfall events occuring within the 7 days prior to the avalanche are retained, with distance to the closest AR recorded. Distances of zero indicate the avalanche occurred within the AR polygon boundary. Attributes of AR duration and strength are joined to the avalanche database table and exported as a CSV. Any NA values in the resulting table indicate that no AR event occurred within 7 days prior to the avalanche.



#### Setup:

In [2]:
import pandas as pd
import geopandas as gpd
from datetime import timedelta

from config import landfall_events_fp, ak_pts, avy_db

In [3]:
#read in AK points reference, avalanche db, and AR events shp
ak_pts = pd.read_csv(ak_pts)
avy = pd.read_csv(avy_db, skiprows=1)
ars = gpd.read_file(landfall_events_fp)

In [4]:
#compute duration of each event and event intensity (strength/duration)
ars['start'] = pd.to_datetime(ars['start'])
ars['end'] = pd.to_datetime(ars['end'])

for i in ars.index:
    #after subtracting, add 6hrs as minimum event length.... this assures a single timestep event is not zero duration!
    ars.loc[i, 'Duration_hrs'] = ((ars['end'][i]-ars['start'][i]).total_seconds()/3600) + 6

ars['AR_Total_Intensity'] = ars['sumtot_str']/ars['Duration_hrs']
ars['AR_Relative_Intensity'] = ars['sumrel_str']/ars['Duration_hrs']

In [5]:
#join avalanche db to AK points, clean table for processing, and add geometry using lat/lon info
df = avy.merge(ak_pts, how='left', left_on='Loc', right_on='name')
df['date'] = pd.to_datetime(df['Avalanche Date'])
df.rename(columns={'Avalanche ID (new)' : 'ID'}, inplace=True)
df = df[['ID', 'date', 'Trigger', 'name', 'latitude', 'longitude']]
df['geometry'] = gpd.GeoSeries.from_xy(df['longitude'], df['latitude'], crs=ars.crs)
df = gpd.GeoDataFrame(df, geometry='geometry')

/tmp/ipykernel_1570/107876706.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['Avalanche Date'])


In [7]:
#convert both to 3338 for minimum distance measurement
df_3338 = df.to_crs('EPSG:3338')
ars_3338 = ars.to_crs('EPSG:3338')

#### Temporal filtering:

In [8]:
#create a duration dictionary with AR ids and their start and end dates
ids = ars_3338['event_id']
starts = ars_3338['start']
ends = ars_3338['end']

ars_dict = {i : [s, e] for i, s, e in zip(ids, starts, ends)}


In [9]:
#~2 min
#for each avalanche ID, create a date range of 1 week preceding the avalanche
#then for each AR in the duration dictionary, compare the AR date range to the avalanche date range
#if overlapping, add the avalanche ID and list of one or more ARs to a new dictionary

avy_ars = {}

for avy in df_3338['ID'].unique():

    #create a range of dates from 1 week before avalanche to avalanche
    avy_end_date = df_3338[df_3338['ID']==avy]['date']
    avy_start_date = avy_end_date - timedelta(days=7)

    ars_sub = []

    #check if the avalanche date range overlaps any of the AR ranges
    for k in ars_dict.keys():
        ar_start_date = ars_dict[k][0]
        ar_end_date = ars_dict[k][1]

        #add to list if overlapping
        if (avy_start_date <= ar_end_date).bool() & (ar_start_date <= avy_end_date).bool():
            ars_sub.append(k)

    #if any ARs overlap, add list to dictionary    
    if len(ars_sub) > 0:
        avy_ars[avy] = [ars_sub]

#### Spatial Analysis:

In [10]:
#using the dictionary of ARs occuring within 1 week prior to avalanche, find the AR with minimum distance to the AR
#record the AR id and its distance from avalanche (in km) in a new dict
#distance values of 0 indicate the point is inside the AR polygon
#if more than one AR have the same distance of zero, only the first is retained
 
avy_ars_min_dist = {}

for a in avy_ars:
    pt_a = df_3338.loc[df_3338['ID']==a].geometry

    dists = []

    for b in avy_ars[a][0]:
        pt_b = ars_3338.iloc[[b]].geometry
        dist = pt_a.distance(pt_b, align=False)
        dists.append(round(dist.iloc[0]/1000)) #convert m to km, and round to integer
    
    avy_ars_min_dist[a] = [avy_ars[a][0][pd.Series(dists).idxmin()], dists[pd.Series(dists).idxmin()]]    

In [11]:
#convert minimum distance dict to dataframe and join to original avalanche points
#then join AR properties
result = df_3338.merge(
    pd.DataFrame.from_dict(avy_ars_min_dist, orient='index', columns=['AR_ID', 'min_km']),
     how='left', left_on='ID', right_index=True).merge(
        ars_3338, how='left', left_on='AR_ID', right_on='event_id')

#clean results table and compute intensity using hours (strength over time)
result = result.rename(columns={'ID':'Avalanche_ID',
'date':'Date', 
'name':'Location',
'latitude':'Latitude',
'longitude':'Longitude', 
'AR_ID':'AR_Event_ID', 
'min_km':'Avalanche_to_AR_Distance_km',
'start':'Start',
'end':'End',
'sumtot_str':'Cumulative_AR_Total_Strength',
'sumrel_str':'Cumulative_AR_Relative_Strength'
}).drop(columns=['event_id', 'geometry_x', 'geometry_y'])

In [12]:
result

,Avalanche_ID,Date,Trigger,Location,Latitude,Longitude,AR_Event_ID,Avalanche_to_AR_Distance_km,Start,End,Cumulative_AR_Total_Strength,Cumulative_AR_Relative_Strength,Duration_hrs,AR_Total_Intensity,AR_Relative_Intensity
0,5,1988-04-09,Natural,Anchorage,61.1817,-149.9930,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1,18,1992-11-08,Natural,Anchorage,61.1817,-149.9930,44.0,0.0,1992-11-08 00:00:00,1992-11-09 06:00:00,8038923.0,2162.0,36.0,223303.416667,60.055556
2,20,1993-01-10,Human,Anchorage,61.1817,-149.9930,53.0,0.0,1992-12-26 12:00:00,1993-01-06 18:00:00,37830618.0,8651.0,276.0,137067.456522,31.344203
3,42,1999-03-26,no data,Anchorage,61.1817,-149.9930,396.0,0.0,1999-03-23 00:00:00,1999-03-25 00:00:00,4392385.0,775.0,54.0,81340.462963,14.351852
4,66,2000-11-11,Human,Anchorage,61.1817,-149.9930,486.0,0.0,2000-11-03 00:00:00,2000-11-11 12:00:00,42936833.0,7400.0,210.0,204461.109524,35.238095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,128,2010-02-13,no data,Whittier,60.7730,-148.6840,1006.0,0.0,2010-02-05 06:00:00,2010-02-06 12:00:00,3368417.0,1178.0,36.0,93567.138889,32.722222
165,130,2010-02-18,no data,Moose Pass,60.4876,-149.3670,1007.0,0.0,2010-02-10 00:00:00,2010-02-11 18:00:00,1544453.0,532.0,48.0,32176.104167,11.083333
166,132,2010-05-29,no data,Petersville,62.3730,-150.7332,1026.0,64.0,2010-05-29 00:00:00,2010-05-29 00:00:00,390579.0,236.0,6.0,65096.500000,39.333333
167,158,2015-04-01,Human,Chitina,61.5158,-144.4370,1291.0,204.0,2015-03-26 00:00:00,2015-03-30 18:00:00,13891310.0,3682.0,120.0,115760.916667,30.683333


In [13]:
print("There are " + str(len(df_3338['ID'].unique())) + " unique avalanche IDs in the revised database...")
print(str(len(avy_ars)) + " of those avalanches have at least one landfalling AR occuring within the 7 days preceding the avalanche...")

There are 169 unique avalanche IDs in the revised database...
149 of those avalanches have at least one landfalling AR occuring within the 7 days preceding the avalanche...
